In [7]:
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, average_precision_score,precision_score,f1_score,recall_score

# create confusion matrix
y_true = np.array([-1]*70 + [0]*160 + [1]*30)
y_pred = np.array([-1]*40 + [0]*20 + [1]*20 + 
                  [-1]*30 + [0]*80 + [1]*30 + 
                  [-1]*5 + [0]*15 + [1]*20)
cm = confusion_matrix(y_true, y_pred)
conf_matrix = pd.DataFrame(cm, index=['Cat','Dog','Pig'], columns=['Cat','Dog','Pig'])

print(y_true.shape)
print(y_pred.shape)
print('------Weighted------')
print('Weighted precision', precision_score(y_true, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_true, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_true, y_pred, average='weighted'))
print('------Macro------')
print('Macro precision', precision_score(y_true, y_pred, average='macro'))
print('Macro recall', recall_score(y_true, y_pred, average='macro'))
print('Macro f1-score', f1_score(y_true, y_pred, average='macro'))
print('------Micro------')
print('Micro precision', precision_score(y_true, y_pred, average='micro'))
print('Micro recall', recall_score(y_true, y_pred, average='micro'))
print('Micro f1-score', f1_score(y_true, y_pred, average='micro'))

(260,)
(260,)
------Weighted------
Weighted precision 0.6314062748845358
Weighted recall 0.5576923076923077
Weighted f1-score 0.575114540631782
------Macro------
Macro precision 0.5193926846100759
Macro recall 0.589781746031746
Macro f1-score 0.5233019853709507
------Micro------
Micro precision 0.5576923076923077
Micro recall 0.5576923076923077
Micro f1-score 0.5576923076923077


In [ ]:
a = [1,2,3]
b = a + [4]
b = tuple(b)
b

(1, 2, 3, 4)

: 

In [ ]:
import torch

ckp_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/mil-methods/output-model/output-test/resnet1-meanmil-ngc-customsplit/fold_0_model_best_auc.pt'
torch.load(ckp_path)
ckp_dict = torch.load(ckp_path)

In [ ]:
import csv
wsis = []
file_name = './simclr/train_label.csv'
with open(file_name, 'r') as f:
    lines = f.readlines()
    for row in lines:
        row = row.strip().split(',')
        wsis.append(row[0])
print(wsis)        


['BJXFK-LA-591455', 'c00567195', 'BJXFK-XXBA-579873', 'BJXFK-XA-575822', 'c00564281', 'BJXK-XIMEA-FFK-P-599781', 'BJXFK-YX577236', 'F22', 'BJXFK-XA576163', 'BJFFK-SCLC574253', 'C00564517', 'BJFFK-XA574864', 'BJFFK-XA573779', 'BJFFK-XA574570', 'BJXK-XIMEA-FFK-P-592670', 'BJXFK-YX575769', 'BJFFK-XA575058', 'F2', 'C00570399', 'BJXK-XIMEA-FFK-P-601186', 'BJXK-XIMEA-FFK-P-599581', 'BJXFK-XA577091', 'BJXFK-XA575834', 'BJXFK-XA583281', 'c00568834', 'F25', 'BJXFK-YX-576622', 'BJXFK-XA576692', 'BJFFK-XA571457', 'BJXFK-XA577014', 'c00569478', 'BJXFK-YX583088', 'BJXFK-XA577016', 'BJFFK-XA571327', 'C00569905', 'c00568308', 'BJFFK-SCLC572374', 'BJFFK-XA571332', 'c00567186', 'c00568523', 'BJFFK-XA572378', 'BJFFK-XA575640', 'BJXFK-YX584459', 'BJFFK-XA575740', 'BJXK-XIMEA-FFK-P-601001', 'BJXFK-XA-581051', 'BJXK-XIMEA-FFK-P-593609', 'c00568309', 'BJFFK-XA571956', 'BJFFK-XA570908', 'BJXK-XIMEA-FFK-P-602284', 'C00562488', 'BJXFK-XA579315', 'BJXK-XIMEA-FFK-P-598709', 'BJFFK-XA573780', 'BJFFK-XA571951', 'F

In [4]:
import os
wsi_root = '/home1/wsi/tct/'
sub_paths = [
            'NILM',
            'POS'
        ]
wsi_dirs = []
for sub_path in sub_paths:
    wsi_dirs.extend([os.path.join(wsi_root, sub_path, wsi_name) for wsi_name in os.listdir(os.path.join(wsi_root, sub_path))])
# data_roots = list(map(lambda x: os.path.join(wsi_root, x), sub_paths))
# data_roots
# wsi_dirs = [os.path.join(wsi_root, sub_dir) for sub_dir in os.listdir(data_roots) ]
wsi_dirs

['/home1/wsi/tct/NILM/xCR20018818-NILM',
 '/home1/wsi/tct/NILM/xCY20005897-NILM',
 '/home1/wsi/tct/NILM/xCY20006000-NILM',
 '/home1/wsi/tct/NILM/xCY20006569-NILM',
 '/home1/wsi/tct/NILM/CX20179985-NILM',
 '/home1/wsi/tct/NILM/CX20166624-NILM',
 '/home1/wsi/tct/NILM/CX20175637-NILM',
 '/home1/wsi/tct/NILM/xCR20018330-NILM',
 '/home1/wsi/tct/NILM/xCY20008034-NILM',
 '/home1/wsi/tct/NILM/xCY20006621-NILM',
 '/home1/wsi/tct/NILM/xCR20018882-NILM',
 '/home1/wsi/tct/NILM/xCY20005843-NILM',
 '/home1/wsi/tct/NILM/CX20179988-NILM',
 '/home1/wsi/tct/NILM/xCR20018485-NILM',
 '/home1/wsi/tct/NILM/xCR20018268-NILM',
 '/home1/wsi/tct/NILM/xCY20006943-NILM',
 '/home1/wsi/tct/NILM/CX20152317-NILM',
 '/home1/wsi/tct/NILM/xCY20007765-NILM',
 '/home1/wsi/tct/NILM/CX20171594-NILM',
 '/home1/wsi/tct/NILM/202000822',
 '/home1/wsi/tct/NILM/CX20176960-NILM',
 '/home1/wsi/tct/NILM/xCY20006719-NILM',
 '/home1/wsi/tct/NILM/xCY20006691-NILM',
 '/home1/wsi/tct/NILM/xCY20007597-NILM',
 '/home1/wsi/tct/NILM/CX201749

In [131]:
from torchvision import transforms
import glob

train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                          transforms.RandomHorizontalFlip(p=0.5),
                                          transforms.ToTensor()])
    

train_set = Whole_Slide_Patchs_Ngc(
            data_dir='/home1/wsi/ngc-2023-1333/',
            train_label_path='/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/ngc_labels/ngc_train_label.csv',
            transform=train_transform
        )


In [77]:
import numpy as np
import torch
import torch.nn.functional as f
from sklearn.metrics import roc_curve, roc_auc_score

n_classes = 5
y_label = np.random.randint(0,n_classes, (10))
y_pred = np.random.rand(10, n_classes)
y_pred = torch.softmax(torch.tensor(y_pred), dim=1)
y_label_one_hot = np.eye(n_classes)[y_label]
print(y_label_one_hot, y_pred)
roc_auc = dict()

for i in range(n_classes):
    roc_auc[i] = roc_auc_score(y_label_one_hot[:, i], y_pred[:, i])
roc_auc = list(roc_auc.values())
roc_auc_macro = np.mean(roc_auc)
roc_auc_macro

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]] tensor([[0.2765, 0.1852, 0.2274, 0.1506, 0.1602],
        [0.2159, 0.1409, 0.1849, 0.2888, 0.1695],
        [0.1244, 0.1427, 0.2141, 0.2983, 0.2204],
        [0.1451, 0.2369, 0.2650, 0.2379, 0.1151],
        [0.1354, 0.1046, 0.2688, 0.2269, 0.2644],
        [0.2930, 0.1570, 0.2331, 0.1355, 0.1814],
        [0.2257, 0.3268, 0.1634, 0.1427, 0.1414],
        [0.2123, 0.1062, 0.2431, 0.2417, 0.1966],
        [0.2173, 0.2060, 0.2000, 0.2304, 0.1462],
        [0.1517, 0.2533, 0.1651, 0.2723, 0.1577]], dtype=torch.float64)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
a = [1,2,3]
b = a + [4]
b = tuple(b)
b

(1, 2, 3, 4)

: 

In [34]:
import torch
from torchvision import transforms

class args_class():
    def __init__(self,):
        self.data_dir = '/home/huangjialong/projects/SimCLR-CIFAR10/data'
        
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor()])
args = args_class()
train_set = CIFAR10Pair(root=args.data_dir,
                        train=True,
                        transform=train_transform,
                        download=True)    

Files already downloaded and verified


In [37]:
train_set[10][0].shape

torch.Size([2, 3, 224, 224])

In [36]:
train_set[0][1]

6

In [ ]:
import torch

tensor = torch.tensor([1.0, -2.0, 3.0, -4.0])
sign_tensor = torch.sign(tensor)

print(sign_tensor)  # 输出: tensor([1., -1., 1., -1.])

tensor([ 1., -1.,  1., -1.])


In [24]:
import os
root_dir = '/home1/wsi/ngc-2023-224/'
sub_dirs = [
    'Unannotated_KSJ/Unannotated-KSJ-TCTNGC-NILM',
    'Unannotated_KSJ/Unannotated-KSJ-TCTNGC-POS',
    'Unannotated_XIMEA/Unannotated-XIMEA-TCTNGC-NILM',
    'Unannotated_XIMEA/Unannotated-XIMEA-TCTNGC-POS'
]
sum_wsi = 0
for sub_dir in sub_dirs:
    sub_dir_path = os.path.join(root_dir, sub_dir)
    sum_wsi += len(os.listdir(sub_dir_path))
sum_wsi
sum_images = 0

def count_fun(sub_dir):
    sum_images = 0
    input_dir_tmp = os.path.join(root_dir, sub_dir)
    for root, dirs, files in os.walk(input_dir_tmp):
        for filename in files:
            if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.jpeg'):
                sum_images += 1
    return sum_images 
for sub_dir in sub_dirs:
    sum_images += count_fun(sub_dir)
print(sum_images)

565720


In [2]:
from torch import tensor
from torchmetrics.classification import BinarySpecificity
target = tensor([0, 1, 0, 1, 0, 1])
preds = tensor([0, 0, 1, 1, 0, 1])
metric = BinarySpecificity()
metric(preds, target)


tensor(0.6667)

In [1]:
import torch

path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/simclr/output-model/new-simclr-puc/biomed_simclr_infonce_filterGC_224_4*256/biomed_simclr_infonce_filterGC_224_4*256_epoch10.pt'
tensor = torch.load(path)
tensor

{'adapter': OrderedDict([('hide_layer.weight',
               tensor([[ 0.0109, -0.0350,  0.0311,  ...,  0.0307,  0.0095,  0.0083],
                       [-0.0034,  0.0327,  0.0025,  ..., -0.0300,  0.0202,  0.0268],
                       [ 0.0229,  0.0451,  0.0338,  ...,  0.0130, -0.0130,  0.0236],
                       ...,
                       [-0.0213,  0.0386,  0.0194,  ..., -0.0484, -0.0293, -0.0139],
                       [ 0.0197,  0.0509,  0.0287,  ..., -0.0375,  0.0119,  0.0043],
                       [ 0.0115,  0.0238,  0.0194,  ...,  0.0040, -0.0067, -0.0480]],
                      device='cuda:0')),
              ('hide_layer.bias',
               tensor([-0.2269,  0.2048,  0.1236,  0.0651,  0.0013,  0.0025,  0.1936,  0.0200,
                        0.2058,  0.2394,  0.2005,  0.4795,  0.3440, -0.0141,  0.6812,  0.0355,
                        0.5717, -0.5975,  0.0265, -0.0436, -0.1840, -0.1434,  0.1415, -0.0168,
                       -0.2090,  0.0154,  0.0908,  0.0

In [2]:
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = open_clip.get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

In [3]:
from PIL import Image

model.cuda()
img_path = '/home1/wsi/gc-224/NILM/202000767/1_10.jpg'
input = preprocess_val(Image.open(img_path)).unsqueeze(0).to('cuda')
img_feature, _, scalar = model(input)
img_feature.shape

img_feature1 = model.encode_image(input)
img_feature1.shape

img_feature2 = model.visual.trunk(input)

img_feature2 /= img_feature


In [11]:
from torchsummary import summary
input_size = (3, 224, 224)
summary(model.to('cuda'), input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
          Identity-2             [-1, 196, 768]               0
        PatchEmbed-3             [-1, 196, 768]               0
           Dropout-4             [-1, 197, 768]               0
          Identity-5             [-1, 197, 768]               0
          Identity-6             [-1, 197, 768]               0
         LayerNorm-7             [-1, 197, 768]           1,536
            Linear-8            [-1, 197, 2304]       1,771,776
          Identity-9          [-1, 12, 197, 64]               0
         Identity-10          [-1, 12, 197, 64]               0
          Dropout-11         [-1, 12, 197, 197]               0
           Linear-12             [-1, 197, 768]         590,592
          Dropout-13             [-1, 197, 768]               0
        Attention-14             [-1, 1

In [19]:
import torch.nn.functional as F
import torch

def fix_seed(seed):
    torch.manual_seed(seed) # 为CPU设置随机种子
    torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
    torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

fix_seed(2024)
# Example of target with class indices
input = torch.randn(5, 3, requires_grad=True)
target = torch.randint(3, (5,), dtype=torch.int64)
print(input, target)
loss = F.cross_entropy(input, target)
loss

tensor([[-0.0404,  1.7260, -0.8140],
        [ 1.3722,  0.5060, -0.4823],
        [-0.7853,  0.6681, -0.4439],
        [ 0.1888,  0.5986,  0.6458],
        [ 0.6306, -1.4668, -0.6798]], requires_grad=True) tensor([0, 2, 2, 2, 1])


tensor(1.8474, grad_fn=<NllLossBackward0>)

In [23]:
import torch.nn.functional as F

target = torch.tensor([0, 2, 2, 2, 1])
target = F.one_hot(target, num_classes=3)
target

TypeError: one_hot() got an unexpected keyword argument 'dtype'

In [36]:
import torch
a = "1"
b = int(1)
c = torch.tensor(b)
c.dtype

torch.int64

In [39]:
input = torch.tensor([[-0.0404,  1.7260, -0.8140],
        [ 1.3722,  0.5060, -0.4823],
        [-0.7853,  0.6681, -0.4439],
        [ 0.1888,  0.5986,  0.6458],
        [ 0.6306, -1.4668, -0.6798]], dtype=torch.float32)
target = torch.tensor([0, 2, 2, 2, 1])
loss = F.cross_entropy(input, target)
loss

tensor(1.8474)

In [37]:
import torch.nn.functional as F
import torch

input = torch.tensor([[-0.0404,  1.7260, -0.8140],
        [ 1.3722,  0.5060, -0.4823],
        [-0.7853,  0.6681, -0.4439],
        [ 0.1888,  0.5986,  0.6458],
        [ 0.6306, -1.4668, -0.6798]], dtype=torch.float32)
target = torch.tensor([0, 2, 2, 2, 1])
target = F.one_hot(target, num_classes=3).type(torch.float32)
loss = F.cross_entropy(input, target)
loss


tensor(1.8474)